## 0.0 Importar Pacotes

In [2]:
from matplotlib import gridspec
from datetime import datetime
from ipywidgets import fixed
from tabulate import tabulate
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings
import tqdm
import ipywidgets as widgets
import seaborn as sns
import matplotlib.ticker as mtick
import plotly.express as px

## 0.1 Funções Auxiliares

In [3]:
pd.options.display.float_format = '{:.2f}'.format

plt.rcParams['figure.figsize'] = (11, 7)

warnings.filterwarnings('ignore')

# 1.0 Dados

## 1.1 Importando Dados

In [4]:
df = pd.read_csv('Datasets/kc_house_data.csv')

## 1.2 Dimensão dos Dados

In [5]:
print('Número de linhas é de :', df.shape[0])
print('Número de colunas é de :', df.shape[1])

Número de linhas é de : 21613
Número de colunas é de : 21


## 1.3 Tipos de Variáveis

In [ ]:
df.dtypes

## 1.4 Alterando os tipos de variáveis

In [ ]:
# Alterando a data para o formato de Datetime
df['date'] = pd.to_datetime(df['date'])

# Modificando a coluna yr_renovated para Datetime
df['yr_renovated'] = df['yr_renovated'].replace(0, pd.NaT)
df['yr_renovated'] = pd.to_datetime(df['yr_renovated'], format='%Y')

## 1.5 Criação de novas features

In [ ]:
# Criação de coluna para informar se foi construído antes ou depois de 1955
df['bf1955'] = df['yr_built'].apply(lambda x: '>1955' if x > 1955
                                    else '<1955')
# Criação de coluna para informar se possuí porão ou não.
df['basement'] = df['sqft_basement'].apply(lambda x: 'no' if x == 0
                                           else 'yes')
# Criação de colunas para mês e ano.
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year

# Criação de uma nova coluna para informar se imóvel foi reformado ou não
df['renovated'] = df['yr_renovated'].apply(
    lambda x: 'yes' if pd.notnull(x) else 'No')

# Criação de uma nova coluna para informar condição do imóvel
df['condition'] = df['condition'].astype(int)
df['condition_type'] = df['condition'].apply(
    lambda x: 'bad' if x <= 2 else 'regular' if (x == 3) | (x == 4) else 'good')

# Criação de nova coluna para informar estação do ano
df['season'] = df['month'].apply(lambda x: 'summer' if (x > 5) & (x < 8) else
                                           'spring' if (x > 2) & (x < 5) else
                                           'fall' if (x > 8) & (x < 12) else
                                           'winter')
df.head()

## 1.5 Limpeza de Dados

## 1.5.1 Verificando Duplicadas

In [ ]:
#Verficando se existem linhas duplicadas através do ID do DataFrame
duplicated_column = df['id'].duplicated()
print(duplicated_column.value_counts())

In [ ]:
#Realizando a exclusão das linhas duplicadas e verificando se foram corretamente excluídas
df = df.drop_duplicates(subset=['id'], keep = 'last')
new_duplicated_column = df['id'].duplicated()
print(new_duplicated_column.value_counts())

## 1.5.2 Valores Nulos

In [ ]:
df.isna().sum()

## 1.5.3 Análise Estatística

In [ ]:
df.columns

In [ ]:
df[['price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'sqft_above', 'sqft_basement', 'yr_built','sqft_living15', 'sqft_lot15']].describe().T

In [ ]:
bed = df[df['bedrooms']==33].index
df = df.drop(bed)
df['bedrooms'].unique()

# 2.0 Exploração de Dados

## 2.1 Correlação

In [ ]:
matrix = np.triu(df.corr())

cmap = sns.diverging_palette(250, 10, as_cmap=True)
ax = sns.heatmap(df.corr(), cmap = cmap, square = True, cbar_kws={"shrink": .6}, annot = False, center = 0, mask = matrix);

# Rotacione as legendas dos eixos x e y em 45 graus
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')

# Adicione uma legenda
cbar = ax.collections[0].colorbar
cbar.set_label('Correlação')

plt.subplots_adjust(bottom=0.15, left=0.15)
plt.tight_layout()        

# 3.0 Insights

In [ ]:
df =  df.copy()

In [ ]:
df.dtypes

## H1: Imóveis com a vista para a água são peloo menos 30% mais caros.

In [ ]:
h1 = df[['price', 'waterfront']].groupby('waterfront').mean().reset_index()
sns.barplot(x = h1['waterfront'], y = h1['price'])
plt.ticklabel_format(style='plain', axis='y')
plt.xticks(h1['waterfront'], ['Sem','Com'])
plt.title('Média de valores para locais com e sem vista para a água')
plt.xlabel('Vista para a água')
plt.ylabel('Preço médio')


h1['price_divided'] = h1['price'] / h1['price'].shift()
h1.head()

## H2: Imóveis com data de construção menor que 1955 são em média 50% mais baratos

In [ ]:
h2 = df[['bf1955', 'price']].groupby('bf1955').mean().reset_index()

sns.barplot(x = h2['bf1955'], y = h2['price'])
plt.ticklabel_format(style='plain', axis='y')

plt.title('Média de valores para locais com construção anterior a 1955')
plt.ylabel('Preço médio')


h2['price_divided'] = h2['price'] / h2['price'].shift()
h2.head()

## H3: Imóveis sem porão poussem área total (sqrt_lot) pelo menos 50% maiores do que imóveis com porão

In [ ]:
h3 = df[['basement','sqft_lot']].groupby('basement').mean().reset_index()

sns.barplot(x = h3['basement'], y = h3['sqft_lot'])
plt.ticklabel_format(style='plain', axis='y')

plt.title('Média da área total de imóveis com e sem porão')
plt.ylabel('Área média')


h3['sqft_proportion'] = h3['sqft_lot'] / h3['sqft_lot'].shift()
h3.head()

## H4: O crescimento do preço dos imóveis ano após ano (YoY) é de 10%

In [ ]:
h4 = df[['year', 'price']].groupby('year').mean().reset_index()

sns.barplot(x = h4['year'], y = h4['price'])
plt.ticklabel_format(style='plain', axis='y')

plt.title('Média do valor de imóveis nos anos de 2014 e 2015')
plt.ylabel('Preço médio')
plt.xlabel('Ano')


h4['price_divided'] = h4['price'] / h4['price'].shift()
h4.head()

## H5: Imóveis com 3 banheiros tem um crescimento MoM de 15%

In [ ]:
h5 = df[df['bathrooms']==3]
h5 = h5[['month','price']].groupby('month').mean().reset_index()

sns.lineplot(data = h5, x = 'month',y='price', markers = True, color = 'b')
plt.title('Média do valor de imóveis ao passar dos meses')
plt.ylabel('Preço médio')
plt.xlabel('Meses')

h5['Diferenca_Perc'] = h5['price'].pct_change() * 100
h5.head()

## H6: Imóveis com data de construção inferior a 1955 e  reformados são 50% mais baratos, comparado aos não reformados

In [ ]:
h6 = df[df['bf1955'] == '<1955']
h6 = h6[['renovated','price']].groupby(['renovated']).mean().reset_index()

sns.barplot(x = h6['renovated'], y = h8['price'])
plt.ticklabel_format(style='plain', axis='y')

plt.title('Média do valor de imóveis de construção antes de 1955 com e sem reforma')
plt.ylabel('Preço médio')
plt.xlabel('Reformados')


price_divided = h6['price'].shift() / h6['price']
print(price_divided)
#O imóvel é 30% mais barato

## H7: Imóveis em boas condições e com vista para água são 20% mais caros em comparação aos em más condições e com vista para água

In [ ]:
h7 = df[['price','waterfront','condition_type']].groupby(['waterfront','condition_type']).mean().reset_index()

good_water = h7[(h7['condition_type']=='good') & (h7['waterfront']==1)].reset_index()
bad_water =h7[(h7['condition_type']=='bad') & (h7['waterfront']==1)].reset_index()

print(good_water['price']/bad_water['price'])

fig = plt.figure(figsize=(24,12))
specs = gridspec.GridSpec( ncols=2, nrows=1, figure=fig )

ax1 =fig.add_subplot( specs[0, 0] ) # First Row
ax2 =fig.add_subplot( specs[0,1] )

ax1.bar(good_water['condition_type'], good_water['price'])
ax2.bar(bad_water['condition_type'], bad_water['price'])

print(good_water['price'])
print(bad_water['price'])

## H8: Imóveis em boas condições e com vista para água são pelo menos 10% mais caros em comparação aos em boas condições e sem vista para água

In [ ]:
h8 = df[df['condition_type'] == 'good']

h8 = h8[['price','waterfront']].groupby('waterfront').mean().reset_index()

sns.barplot(x = h8['waterfront'], y = h8['price'])
plt.ticklabel_format(style='plain', axis='y')
plt.xticks(h8['waterfront'], ['Sem','Com'])
plt.title('Média do valor de imóveis em boas condições com e sem vista para água')
plt.ylabel('Preço médio')
plt.xlabel('Vista para a água')


h8['price_divided'] = h8['price'] / h8['price'].shift()

h8.head()

## H9: Imóveis que nunca foram reformados são pelo menos 10% mais baratos que os não reformados

In [ ]:
h9 = df[['renovated','price']].groupby('renovated').mean().reset_index()

sns.barplot(x = h9['renovated'], y = h9['price'])
plt.ticklabel_format(style='plain', axis='y')

plt.title('Média do valor de imóveis reformados')
plt.ylabel('Preço médio')
plt.xlabel('Reformado')


print(h9['price'].shift()/h9['price'])

## H10: Imóveis construídos antes de 1955 possuem mais banheiros

In [ ]:
h10 = df[['bathrooms','bf1955']].groupby('bf1955').mean()


print(h10)


# 3.0 Questão de Negócio

## 3.1 Quais são os imóveis que a House Rocket deveria comprar e por qual preço ?

In [ ]:
df = df.copy()

df1 = df[['zipcode','price']].groupby('zipcode').median().reset_index()

df2 = pd.merge(df1,df, on='zipcode', how = 'inner')

df2 = df2.rename(columns = {'price_y':'price', 'price_x':'price_median'})

for i, row in df2.iterrows():
    if (row['price_median']>= row['price']) & (row['condition_type'] == 'good'):
        df2.loc[i,'buy'] = 'yes'
    else:
        df2.loc[i,'buy'] = 'no'

In [ ]:
buy = df2[df2['buy']=='yes']

buy['buy'].shape[0]
buy[['id','price','buy']]

In [ ]:
id_bar = widgets.Dropdown(
        options = df2['buy'].unique().tolist(),
        descriptiono = 'region',
        disabled = False)


In [ ]:
def update_map(df3, id_bar):
    house = df2[(df2['buy'] == id_bar)][['id', 'lat', 'long', 'condition', 'buy', 'price']]
    
    mapa = px.scatter_mapbox(house, lat = 'lat',lon = 'long',
                            size = 'price',
                            color = 'condition',
                            opacity = 0.5,
                            zoom = 10)
    
    mapa.update_layout(mapbox_style = 'open-street-map')
    mapa.update_layout(height = 600, margin = {'b':0, 't':0, 'r':0, 'l': 0})
    mapa.show()
    
widgets.interactive(update_map, df3 = fixed(df2), id_bar = id_bar)

## 3.2 Uma vez comprado, qual é o melhor momento para vendê-lo e por qual preço ?

In [ ]:
df3 = df2.copy()

In [ ]:
df3 = df3[df3['buy']=='yes']

In [ ]:
df4 = df3[['season', 'zipcode', 'price']].groupby(['zipcode', 'season']).median().reset_index()

df4 = df4.rename(columns = {'price' : 'price_m_season', 'season': 'season_median'} ) 

df5 = pd.merge(df3, df4, on='zipcode', how = 'inner')

df5['sale'] = df5.apply(lambda row: row['price'] * 1.1 if row['price_m_season'] > row['price'] else row['price'] * 1.3, axis=1)

df5 = df5.drop_duplicates(subset='id')

In [ ]:
df5[['season', 'price', 'id', 'sale']].head(20)

In [ ]:
sns.barplot(data = df5, x = 'season',y='sale')

# 4.0 Resultados para o negócio

In [ ]:
df5['profit'] = df5['sale'] - df5['price']

In [ ]:
print("Com a compra e venda dos imóveis a empresa irá lucrar {:.2f}".format(df5['profit'].sum()))

# 5.0 Resultados dos teste das hipóteses

In [ ]:
hipoteses = [['Hipótese','Validação','Resultado'],
            ['H1','Verdadeiro','Os imóveis com vista para a água são em média 212% mais caros.'],
            ['H2','Falso','O ano de construção dos imóveis não parece influenciar diretamente no preço.'],
            ['H3','Falso','Os imóveis sem porão possuem área total maior, porém em média de 20% maior.'],
            ['H4','Falso','O crescimento da média de preço dos imóveis é inferior a 1%.'],
            ['H5','Falso','O preço dos imóveis varia de acordo com o mês, em momentos ocorrendo um aumento em outros uma redução.'],
            ['H6','Falso','Os imóveis com data de construção inferior a 1955 e reformados são na verdade 30% mais baratos.'],
            ['H7','Falso','Os imóveis em más condições e com vista para água são 2% mais caros, comparados aos em boas condições.'],
            ['H8','Verdadeiro','Comparando imóveis em boas condições, os que possuem vista para água são em média 163% mais caros.'],
            ['H9','Verdadeiro','Os imóveis não reformado são em média 30% mais baratos.'],
            ['H10','Falso','Imóveis após o ano de 1955 possuem mais banheiros.']]


print(tabulate(hipoteses, headers='firstrow'))